In [ ]:
##

In [222]:
import sqlite3
import numpy as np
import pandas as pd 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import recordlinkage
from recordlinkage.preprocessing import clean

In [63]:
# Connect to the Database 
path = "zippedData/im.db"
conn = sqlite3.connect(path)
cursor = conn.cursor()

In [21]:
# to see all the tables in the database.

imdb_df = pd.read_sql(
    """
    SELECT *
    FROM sqlite_master
    """
, conn
)

imdb_df[imdb_df['type'] == 'table']

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [67]:
query1 = """ SELECT * FROM movie_basics ORDER BY -start_year LIMIT 10"""
pd.read_sql(query1, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt5174640,100 Years,100 Years,2115,None,Drama
1,tt5637536,Avatar 5,Avatar 5,2027,None,"Action,Adventure,Fantasy"
2,tt10300398,Untitled Star Wars Film,Untitled Star Wars Film,2026,None,Fantasy
3,tt3095356,Avatar 4,Avatar 4,2025,None,"Action,Adventure,Fantasy"
4,tt10300396,Untitled Star Wars Film,Untitled Star Wars Film,2024,None,None
5,tt6149054,Fantastic Beasts and Where to Find Them 5,Fantastic Beasts and Where to Find Them 5,2024,None,"Adventure,Family,Fantasy"
6,tt10255736,Untitled Marvel Project,Untitled Marvel Project,2023,None,Action
7,tt10298848,Untitled Disney Live-Action Project,Untitled Disney Live-Action Project,2023,None,None
8,tt1757678,Avatar 3,Avatar 3,2023,None,"Action,Adventure,Drama"
9,tt6258542,Wraith of the Umbra and Eidolon II,Wraith of the Umbra and Eidolon II,2023,None,"Adventure,Drama,Fantasy"


In [65]:
# get column info
cursor.execute("PRAGMA table_info(movie_basics)")
cursor.fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'primary_title', 'TEXT', 0, None, 0),
 (2, 'original_title', 'TEXT', 0, None, 0),
 (3, 'start_year', 'INTEGER', 0, None, 0),
 (4, 'runtime_minutes', 'REAL', 0, None, 0),
 (5, 'genres', 'TEXT', 0, None, 0)]

In [23]:
query4 = """ SELECT COUNT(*) FROM movie_basics"""
pd.read_sql(query4, conn)

,COUNT(*)
0,146144


In [30]:
query2 = """ SELECT * FROM movie_ratings LIMIT 10"""
pd.read_sql(query2, conn)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
5,tt1069246,6.2,326
6,tt1094666,7.0,1613
7,tt1130982,6.4,571
8,tt1156528,7.2,265
9,tt1161457,4.2,148


In [25]:
query3 = """ SELECT COUNT(*) FROM movie_ratings"""
pd.read_sql(query3, conn)

,COUNT(*)
0,73856


In [156]:
query6 = """SELECT movie_ratings.movie_id AS imdb_movie_id, movie_basics.primary_title, 
            movie_ratings.averagerating AS average_rating, movie_ratings.numvotes, movie_basics.start_year, 
            movie_basics.runtime_minutes, movie_basics.genres, primary_name AS director
            FROM movie_ratings 
            JOIN movie_basics ON movie_ratings.movie_id=movie_basics.movie_id 
            JOIN (SELECT DISTINCT * FROM directors) AS d ON movie_ratings.movie_id=d.movie_id
            JOIN persons ON persons.person_id=d.person_id
            WHERE movie_ratings.numvotes >=30 AND start_year>2009
            ORDER BY -numvotes
            LIMIT 40"""
pd.read_sql(query6, conn)

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director
0,tt1375666,Inception,8.8,1841066,2010,148.0,"Action,Adventure,Sci-Fi",Christopher Nolan
1,tt1345836,The Dark Knight Rises,8.4,1387769,2012,164.0,"Action,Thriller",Christopher Nolan
2,tt0816692,Interstellar,8.6,1299334,2014,169.0,"Adventure,Drama,Sci-Fi",Christopher Nolan
3,tt1853728,Django Unchained,8.4,1211405,2012,165.0,"Drama,Western",Quentin Tarantino
4,tt0848228,The Avengers,8.1,1183655,2012,143.0,"Action,Adventure,Sci-Fi",Joss Whedon
5,tt0993846,The Wolf of Wall Street,8.2,1035358,2013,180.0,"Biography,Crime,Drama",Martin Scorsese
6,tt1130884,Shutter Island,8.1,1005960,2010,138.0,"Mystery,Thriller",Martin Scorsese
7,tt2015381,Guardians of the Galaxy,8.1,948394,2014,121.0,"Action,Adventure,Comedy",James Gunn
8,tt1431045,Deadpool,8.0,820847,2016,108.0,"Action,Adventure,Comedy",Tim Miller
9,tt1392170,The Hunger Games,7.2,795227,2012,142.0,"Action,Adventure,Sci-Fi",Gary Ross


In [90]:
query23 = """SELECT COUNT (*)
                FROM (SELECT movie_ratings.movie_id AS imdb_movie_id, movie_basics.primary_title, 
            movie_ratings.averagerating AS average_rating, 
            movie_ratings.numvotes, movie_basics.start_year, movie_basics.runtime_minutes, movie_basics.genres
            FROM movie_ratings 
            JOIN movie_basics ON movie_ratings.movie_id=movie_basics.movie_id )"""
pd.read_sql(query23, conn)

,COUNT (*)
0,73856


In [160]:
query25 = """SELECT COUNT(*)
                FROM (SELECT movie_ratings.movie_id AS imdb_movie_id, movie_basics.primary_title, 
            movie_ratings.averagerating AS average_rating, 
            movie_ratings.numvotes, movie_basics.start_year, movie_basics.runtime_minutes, movie_basics.genres
            FROM movie_ratings 
            JOIN movie_basics ON movie_ratings.movie_id=movie_basics.movie_id 
            WHERE movie_ratings.numvotes >=30 AND start_year>2009)"""
pd.read_sql(query25, conn)

,COUNT(*)
0,43740


'SELECT COUNT(*)\n                FROM (SELECT movie_ratings.movie_id AS imdb_movie_id, movie_basics.primary_title, \n            movie_ratings.averagerating AS average_rating, \n            movie_ratings.numvotes, movie_basics.start_year, movie_basics.runtime_minutes, movie_basics.genres\n            FROM movie_ratings \n            JOIN movie_basics ON movie_ratings.movie_id=movie_basics.movie_id \n            WHERE movie_ratings.numvotes >=30)'

In [123]:
query35 = """SELECT COUNT(*)
                FROM (SELECT movie_ratings.movie_id AS imdb_movie_id, movie_basics.primary_title, 
            movie_ratings.averagerating AS average_rating, movie_ratings.numvotes, movie_basics.start_year, 
            movie_basics.runtime_minutes, movie_basics.genres, primary_name AS director
            FROM movie_ratings 
            JOIN movie_basics ON movie_ratings.movie_id=movie_basics.movie_id 
            JOIN (SELECT DISTINCT * FROM directors) AS d ON movie_ratings.movie_id=d.movie_id
            JOIN persons ON persons.person_id=d.person_id
            WHERE movie_ratings.numvotes >=30)
"""
pd.read_sql(query35, conn)

,COUNT(*)
0,50710


In [189]:
query38 = """
WITH ranked_directors AS (
    SELECT
        d.movie_id,
        p.primary_name AS director_name,
        ROW_NUMBER() OVER (PARTITION BY d.movie_id ORDER BY d.person_id) AS director_rank
    FROM directors d
    JOIN persons p ON d.person_id = p.person_id
    GROUP BY d.movie_id, p.primary_name, d.person_id  -- Ensures distinct directors per movie
)

SELECT 
    mr.movie_id AS imdb_movie_id, 
    mb.primary_title, 
    mr.averagerating AS average_rating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres, 
    MAX(CASE WHEN rd.director_rank = 1 THEN rd.director_name END) AS director1, 
    MAX(CASE WHEN rd.director_rank = 2 THEN rd.director_name END) AS director2
FROM movie_ratings mr
JOIN movie_basics mb ON mr.movie_id = mb.movie_id
LEFT JOIN ranked_directors rd ON mr.movie_id = rd.movie_id
WHERE mr.numvotes >= 30
GROUP BY 
    mr.movie_id, 
    mb.primary_title, 
    mr.averagerating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres
ORDER BY start_year
"""

pd.read_sql(query38, conn)

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director1,director2
0,tt0066787,One Day Before the Rainy Season,7.2,43,2019,114.0,"Biography,Drama",Mani Kaul,None
1,tt0263814,On kadin,7.1,100,2019,NaN,Drama,Serif Gören,None
2,tt0285423,Abolição,7.4,35,2019,NaN,Documentary,Zózimo Bulbul,None
3,tt0437086,Alita: Battle Angel,7.5,88207,2019,122.0,"Action,Adventure,Sci-Fi",Robert Rodriguez,None
4,tt0448115,Shazam!,7.4,109051,2019,132.0,"Action,Adventure,Comedy",David F. Sandberg,None
...,...,...,...,...,...,...,...,...,...
43735,tt6920930,My Sex Robot,4.6,91,2010,NaN,Documentary,None,None
43736,tt7145012,Nofozi,6.3,76,2010,84.0,"Thriller,War",Ahmad Kavari,None
43737,tt7762194,Clownfish,1.0,67,2010,83.0,"Drama,Romance",Charlie Chu,None
43738,tt7782570,Dancing Without You,1.1,67,2010,91.0,Romance,Yun-Chan Lee,None


In [190]:
imdb_df = pd.read_sql(query38, conn)
imdb_df.head()

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director1,director2
0,tt0066787,One Day Before the Rainy Season,7.2,43,2019,114.0,"Biography,Drama",Mani Kaul,None
1,tt0263814,On kadin,7.1,100,2019,NaN,Drama,Serif Gören,None
2,tt0285423,Abolição,7.4,35,2019,NaN,Documentary,Zózimo Bulbul,None
3,tt0437086,Alita: Battle Angel,7.5,88207,2019,122.0,"Action,Adventure,Sci-Fi",Robert Rodriguez,None
4,tt0448115,Shazam!,7.4,109051,2019,132.0,"Action,Adventure,Comedy",David F. Sandberg,None


In [191]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43740 entries, 0 to 43739
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdb_movie_id    43740 non-null  object 
 1   primary_title    43740 non-null  object 
 2   average_rating   43740 non-null  float64
 3   numvotes         43740 non-null  int64  
 4   start_year       43740 non-null  int64  
 5   runtime_minutes  41466 non-null  float64
 6   genres           43619 non-null  object 
 7   director1        43528 non-null  object 
 8   director2        4352 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 3.0+ MB


In [161]:
query128 = """ SELECT COUNT(*)
                FROM (WITH ranked_directors AS (
    SELECT
        d.movie_id,
        p.primary_name AS director_name,
        ROW_NUMBER() OVER (PARTITION BY d.movie_id ORDER BY d.person_id) AS director_rank
    FROM directors d
    JOIN persons p ON d.person_id = p.person_id
    GROUP BY d.movie_id, p.primary_name, d.person_id  -- Ensures distinct directors per movie
)

SELECT 
    mr.movie_id AS imdb_movie_id, 
    mb.primary_title, 
    mr.averagerating AS average_rating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres, 
    MAX(CASE WHEN rd.director_rank = 1 THEN rd.director_name END) AS director1, 
    MAX(CASE WHEN rd.director_rank = 2 THEN rd.director_name END) AS director2
FROM movie_ratings mr
JOIN movie_basics mb ON mr.movie_id = mb.movie_id
LEFT JOIN ranked_directors rd ON mr.movie_id = rd.movie_id
WHERE mr.numvotes >= 30
GROUP BY 
    mr.movie_id, 
    mb.primary_title, 
    mr.averagerating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres)
"""

pd.read_sql(query128, conn)

,COUNT(*)
0,43740


In [ ]:
####################################################################################

In [82]:
query20 = """ SELECT DISTINCT * 
                FROM directors JOIN persons ON directors.person_id=persons.person_id 
                ORDER BY -movie_id
                LIMIT 15"""
pd.read_sql(query20, conn)

,movie_id,person_id,person_id,primary_name,birth_year,death_year,primary_profession
0,tt0285252,nm0899854,nm0899854,Tony Vitale,1964.0,None,"producer,director,writer"
1,tt0462036,nm1940585,nm1940585,Bill Haley,NaN,None,"director,writer,producer"
2,tt0835418,nm0151540,nm0151540,Jay Chandrasekhar,1968.0,None,"director,actor,writer"
3,tt0878654,nm0089502,nm0089502,Albert Pyun,1954.0,None,"director,writer,producer"
4,tt0878654,nm2291498,nm2291498,Joe Baile,NaN,None,"producer,director,camera_department"
5,tt0878654,nm2292011,nm2292011,Howie Askins,NaN,None,"editor,director,cinematographer"
6,tt0879859,nm2416460,nm2416460,Eric Manchester,NaN,None,"director,writer"
7,tt0996958,nm2286991,nm2286991,Tara Cardinal,1978.0,None,"actress,writer,producer"
8,tt0999913,nm0527109,nm0527109,Rod Lurie,1962.0,None,"writer,director,producer"
9,tt10003792,nm10539228,nm10539228,Dinesh Lingden,NaN,None,"director,writer"


In [77]:
query22 = """ SELECT COUNT(*) 
              FROM directors 
              JOIN persons ON directors.person_id = persons.person_id"""
pd.read_sql(query22, conn)

,COUNT(*)
0,291171


In [80]:
query23 = """SELECT COUNT(*)
            FROM (SELECT DISTINCT * 
                FROM directors JOIN persons ON directors.person_id=persons.person_id)
            """
pd.read_sql(query23, conn)

,COUNT(*)
0,163533


In [57]:
query7 = """ SELECT * FROM known_for LIMIT 5"""
pd.read_sql(query7, conn)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [154]:
query30 = """SELECT * 
            FROM known_for JOIN persons ON known_for.person_id=persons.person_id
            WHERE birth_year<>'None'
            ORDER BY birth_year
            LIMIT 10"""

# WHERE movie_id='tt0816692'
pd.read_sql(query30, conn)

,person_id,movie_id,person_id,primary_name,birth_year,death_year,primary_profession
0,nm8572003,tt4099244,nm8572003,Michael Vignola,1.0,NaN,"composer,music_department"
1,nm8572003,tt4600298,nm8572003,Michael Vignola,1.0,NaN,"composer,music_department"
2,nm8572003,tt6417824,nm8572003,Michael Vignola,1.0,NaN,"composer,music_department"
3,nm8572003,tt8200742,nm8572003,Michael Vignola,1.0,NaN,"composer,music_department"
4,nm0613556,tt0043580,nm0613556,Shikibu Murasaki,973.0,1031.0,writer
5,nm0613556,tt0092875,nm0613556,Shikibu Murasaki,973.0,1031.0,writer
6,nm0613556,tt0094100,nm0613556,Shikibu Murasaki,973.0,1031.0,writer
7,nm0613556,tt0256385,nm0613556,Shikibu Murasaki,973.0,1031.0,writer
8,nm0019604,tt1118686,nm0019604,Dante Alighieri,1265.0,1321.0,"writer,soundtrack"
9,nm0019604,tt1598441,nm0019604,Dante Alighieri,1265.0,1321.0,"writer,soundtrack"


In [56]:
query8 = """ SELECT * FROM movie_akas LIMIT 5"""
pd.read_sql(query8, conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [55]:
query9 = """ SELECT * FROM persons LIMIT 5"""
pd.read_sql(query9, conn)

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,None,None,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,None,None,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,None,None,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,None,None,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,None,None,"production_designer,art_department,set_decorator"


In [106]:
query10 = """ SELECT * FROM principals WHERE movie_id='tt1043726'"""
pd.read_sql(query10, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt1043726,10,nm0503600,producer,producer,None
1,tt1043726,1,nm1553725,actor,None,"[""Hercules""]"
2,tt1043726,2,nm4719349,actress,None,"[""Hebe""]"
3,tt1043726,3,nm0012078,actor,None,"[""King Amphitryon""]"
4,tt1043726,4,nm1861467,actress,None,"[""Queen Alcmene""]"
5,tt1043726,5,nm0001317,director,None,None
6,tt1043726,6,nm0393517,writer,None,None
7,tt1043726,7,nm0316417,writer,None,None
8,tt1043726,8,nm1048866,writer,None,None
9,tt1043726,9,nm0203246,producer,producer,None


In [236]:
query100 = """ SELECT * FROM principals WHERE movie_id='tt4912910'"""
pd.read_sql(query100, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt4912910,10,nm1134771,cinematographer,None,None
1,tt4912910,1,nm0000129,actor,None,"[""Ethan Hunt""]"
2,tt4912910,2,nm0147147,actor,None,"[""August Walker""]"
3,tt4912910,3,nm0000609,actor,None,"[""Luther Stickell""]"
4,tt4912910,4,nm0670408,actor,None,"[""Benji Dunn""]"
5,tt4912910,5,nm0003160,director,None,None
6,tt4912910,6,nm0312367,writer,based on the television series created by,None
7,tt4912910,7,nm0009190,producer,producer,None
8,tt4912910,8,nm0616735,producer,producer,None
9,tt4912910,9,nm1154632,composer,None,None


In [244]:
query101 = """
    SELECT movie_basics.primary_title, principals.person_id, principals.category, persons.primary_name
    FROM movie_basics
    JOIN principals ON movie_basics.movie_id=principals.movie_id
    JOIN persons ON principals.person_id=persons.person_id
    WHERE movie_basics.movie_id='tt4912910'
"""
pd.read_sql(query101, conn)

,primary_title,person_id,category,primary_name
0,Mission: Impossible - Fallout,nm0000129,actor,Tom Cruise
1,Mission: Impossible - Fallout,nm0000609,actor,Ving Rhames
2,Mission: Impossible - Fallout,nm0003160,director,Christopher McQuarrie
3,Mission: Impossible - Fallout,nm0009190,producer,J.J. Abrams
4,Mission: Impossible - Fallout,nm0147147,actor,Henry Cavill
5,Mission: Impossible - Fallout,nm0312367,writer,Bruce Geller
6,Mission: Impossible - Fallout,nm0616735,producer,Jake Myers
7,Mission: Impossible - Fallout,nm0670408,actor,Simon Pegg
8,Mission: Impossible - Fallout,nm1134771,cinematographer,Rob Hardy
9,Mission: Impossible - Fallout,nm1154632,composer,Lorne Balfe


In [245]:
query102 = """
    SELECT movie_basics.primary_title, principals.person_id, principals.category, persons.primary_name
    FROM movie_basics
    JOIN principals ON movie_basics.movie_id=principals.movie_id
    JOIN persons ON principals.person_id=persons.person_id
    WHERE movie_basics.movie_id='tt1405365'
"""
pd.read_sql(query102, conn)

,primary_title,person_id,category,primary_name
0,Celeste & Jesse Forever,nm0000704,actor,Elijah Wood
1,Celeste & Jesse Forever,nm0429069,actress,Rashida Jones
2,Celeste & Jesse Forever,nm0566489,writer,Will McCormack
3,Celeste & Jesse Forever,nm0625540,producer,Lee Nelson
4,Celeste & Jesse Forever,nm0731075,actress,Emma Roberts
5,Celeste & Jesse Forever,nm0865189,producer,Jennifer Todd
6,Celeste & Jesse Forever,nm0865297,producer,Suzanne Todd
7,Celeste & Jesse Forever,nm1676221,actor,Andy Samberg
8,Celeste & Jesse Forever,nm1767218,director,Lee Toland Krieger
9,Celeste & Jesse Forever,nm3318928,composer,Zach Cowie


In [246]:
query103 = """
    SELECT movie_basics.primary_title, principals.person_id, principals.category, persons.primary_name
    FROM movie_basics
    JOIN principals ON movie_basics.movie_id=principals.movie_id
    JOIN persons ON principals.person_id=persons.person_id
    WHERE movie_basics.movie_id='tt2226417'
"""
pd.read_sql(query103, conn)

,primary_title,person_id,category,primary_name
0,Insidious: Chapter 2,nm0001347,actress,Barbara Hershey
1,Insidious: Chapter 2,nm0005417,actress,Lin Shaye
2,Insidious: Chapter 2,nm0089658,producer,Jason Blum
3,Insidious: Chapter 2,nm0126284,actress,Rose Byrne
4,Insidious: Chapter 2,nm0502954,cinematographer,John R. Leonetti
5,Insidious: Chapter 2,nm0933940,actor,Patrick Wilson
6,Insidious: Chapter 2,nm1177766,composer,Joseph Bishara
7,Insidious: Chapter 2,nm1191481,writer,Leigh Whannell
8,Insidious: Chapter 2,nm1490123,director,James Wan
9,Insidious: Chapter 2,nm2305431,producer,Oren Peli


In [247]:
query104 = """
    SELECT movie_basics.primary_title, principals.person_id, principals.category, persons.primary_name
    FROM movie_basics
    JOIN principals ON movie_basics.movie_id=principals.movie_id
    JOIN persons ON principals.person_id=persons.person_id
    WHERE movie_basics.movie_id='tt2395427'
"""
pd.read_sql(query104, conn)

,primary_title,person_id,category,primary_name
0,Avengers: Age of Ultron,nm0000375,actor,Robert Downey Jr.
1,Avengers: Age of Ultron,nm0262635,actor,Chris Evans
2,Avengers: Age of Ultron,nm0270559,producer,Kevin Feige
3,Avengers: Age of Ultron,nm0456158,writer,Jack Kirby
4,Avengers: Age of Ultron,nm0498278,writer,Stan Lee
5,Avengers: Age of Ultron,nm0749263,actor,Mark Ruffalo
6,Avengers: Age of Ultron,nm0800209,writer,Joe Simon
7,Avengers: Age of Ultron,nm0923736,director,Joss Whedon
8,Avengers: Age of Ultron,nm1165110,actor,Chris Hemsworth
9,Avengers: Age of Ultron,nm4160687,writer,Jim Starlin


In [248]:
query105 = """
    SELECT movie_basics.primary_title, principals.person_id, principals.category, persons.primary_name
    FROM movie_basics
    JOIN principals ON movie_basics.movie_id=principals.movie_id
    JOIN persons ON principals.person_id=persons.person_id
    WHERE movie_basics.movie_id='tt2469216'
"""
pd.read_sql(query105, conn)

,primary_title,person_id,category,primary_name
0,Her Cry: La Llorona Investigation,nm5393230,composer,Luis Carlos
1,Her Cry: La Llorona Investigation,nm5393603,actor,Ron Gelner
2,Her Cry: La Llorona Investigation,nm5394421,actor,Everardo Guzman
3,Her Cry: La Llorona Investigation,nm5395122,actor,James Ezrin
4,Her Cry: La Llorona Investigation,nm5395150,director,Damir Catic
5,Her Cry: La Llorona Investigation,nm5510714,composer,Art Blackmore
6,Her Cry: La Llorona Investigation,nm5584596,actress,Nichole Ceballos
7,Her Cry: La Llorona Investigation,nm6235651,producer,Hai Nguyen


In [251]:
query106 = """
    SELECT principals.movie_id, principals.person_id, principals.category, persons.primary_name
    FROM principals
    JOIN persons ON principals.person_id=persons.person_id
    WHERE principals.movie_id='tt2469216'

"""
pd.read_sql(query106, conn)

,movie_id,person_id,category,primary_name
0,tt2469216,nm5584596,actress,Nichole Ceballos
1,tt2469216,nm5395122,actor,James Ezrin
2,tt2469216,nm5393603,actor,Ron Gelner
3,tt2469216,nm5394421,actor,Everardo Guzman
4,tt2469216,nm5395150,director,Damir Catic
5,tt2469216,nm6235651,producer,Hai Nguyen
6,tt2469216,nm5510714,composer,Art Blackmore
7,tt2469216,nm5393230,composer,Luis Carlos


In [ ]:
##

In [256]:
query200 = """
    WITH ranked_directors AS (
    SELECT
        d.movie_id,
        p.primary_name AS director_name,
        ROW_NUMBER() OVER (PARTITION BY d.movie_id ORDER BY d.person_id) AS director_rank
    FROM directors d
    JOIN persons p ON d.person_id = p.person_id
    GROUP BY d.movie_id, p.primary_name, d.person_id
),

ranked_principals AS (
    SELECT 
        p.movie_id,
        per.primary_name AS person_name,
        p.category,
        ROW_NUMBER() OVER (PARTITION BY p.movie_id, p.category ORDER BY p.person_id) AS person_rank
    FROM principals p
    JOIN persons per ON p.person_id = per.person_id
    WHERE p.category IN ('actor', 'actress')
)

SELECT 
    mr.movie_id AS imdb_movie_id, 
    mb.primary_title, 
    mr.averagerating AS average_rating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres, 
    MAX(CASE WHEN rd.director_rank = 1 THEN rd.director_name END) AS director1, 
    MAX(CASE WHEN rd.director_rank = 2 THEN rd.director_name END) AS director2,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 1 THEN rp.person_name END) AS actress1,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 2 THEN rp.person_name END) AS actress2,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 3 THEN rp.person_name END) AS actress3,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 4 THEN rp.person_name END) AS actress4,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 1 THEN rp.person_name END) AS actor1,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 2 THEN rp.person_name END) AS actor2,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 3 THEN rp.person_name END) AS actor3,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 4 THEN rp.person_name END) AS actor4
FROM movie_ratings mr
JOIN movie_basics mb ON mr.movie_id = mb.movie_id
LEFT JOIN ranked_directors rd ON mr.movie_id = rd.movie_id
LEFT JOIN ranked_principals rp ON mr.movie_id = rp.movie_id
WHERE mr.numvotes >= 30
GROUP BY 
    mr.movie_id, 
    mb.primary_title, 
    mr.averagerating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres
ORDER BY -mr.numvotes
LIMIT 40
"""

pd.read_sql(query200, conn)

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director1,director2,actress1,actress2,actress3,actress4,actor1,actor2,actor3,actor4
0,tt1375666,Inception,8.8,1841066,2010,148.0,"Action,Adventure,Sci-Fi",Christopher Nolan,None,Ellen Page,None,None,None,Leonardo DiCaprio,Joseph Gordon-Levitt,Ken Watanabe,None
1,tt1345836,The Dark Knight Rises,8.4,1387769,2012,164.0,"Action,Thriller",Christopher Nolan,None,Anne Hathaway,None,None,None,Gary Oldman,Christian Bale,Tom Hardy,None
2,tt0816692,Interstellar,8.6,1299334,2014,169.0,"Adventure,Drama,Sci-Fi",Christopher Nolan,None,Anne Hathaway,Jessica Chastain,Mackenzie Foy,None,Matthew McConaughey,None,None,None
3,tt1853728,Django Unchained,8.4,1211405,2012,165.0,"Drama,Western",Quentin Tarantino,None,Kerry Washington,None,None,None,Leonardo DiCaprio,Jamie Foxx,Christoph Waltz,None
4,tt0848228,The Avengers,8.1,1183655,2012,143.0,"Action,Adventure,Sci-Fi",Joss Whedon,None,Scarlett Johansson,None,None,None,Robert Downey Jr.,Chris Evans,Jeremy Renner,None
5,tt0993846,The Wolf of Wall Street,8.2,1035358,2013,180.0,"Biography,Crime,Drama",Martin Scorsese,None,Margot Robbie,None,None,None,Leonardo DiCaprio,Matthew McConaughey,Jonah Hill,None
6,tt1130884,Shutter Island,8.1,1005960,2010,138.0,"Mystery,Thriller",Martin Scorsese,None,Emily Mortimer,None,None,None,Leonardo DiCaprio,Ben Kingsley,Mark Ruffalo,None
7,tt2015381,Guardians of the Galaxy,8.1,948394,2014,121.0,"Action,Adventure,Comedy",James Gunn,None,Zoe Saldana,None,None,None,Vin Diesel,Bradley Cooper,Chris Pratt,None
8,tt1431045,Deadpool,8.0,820847,2016,108.0,"Action,Adventure,Comedy",Tim Miller,None,Morena Baccarin,None,None,None,Ryan Reynolds,T.J. Miller,Ed Skrein,None
9,tt1392170,The Hunger Games,7.2,795227,2012,142.0,"Action,Adventure,Sci-Fi",Gary Ross,None,Jennifer Lawrence,None,None,None,Stanley Tucci,Josh Hutcherson,Liam Hemsworth,None


In [265]:
query201 = """
    WITH ranked_directors AS (
    SELECT
        d.movie_id,
        p.primary_name AS director_name,
        ROW_NUMBER() OVER (PARTITION BY d.movie_id ORDER BY d.person_id) AS director_rank
    FROM directors d
    JOIN persons p ON d.person_id = p.person_id
    GROUP BY d.movie_id, p.primary_name, d.person_id
),

ranked_principals AS (
    SELECT 
        p.movie_id,
        per.primary_name AS person_name,
        p.category,
        ROW_NUMBER() OVER (PARTITION BY p.movie_id, p.category ORDER BY p.person_id) AS person_rank
    FROM principals p
    JOIN persons per ON p.person_id = per.person_id
    WHERE p.category IN ('actor', 'actress', 'producer')
)

SELECT 
    mr.movie_id AS imdb_movie_id, 
    mb.primary_title, 
    mr.averagerating AS average_rating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres, 
    MAX(CASE WHEN rd.director_rank = 1 THEN rd.director_name END) AS director1, 
    MAX(CASE WHEN rd.director_rank = 2 THEN rd.director_name END) AS director2,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 1 THEN rp.person_name END) AS actress1,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 2 THEN rp.person_name END) AS actress2,
    MAX(CASE WHEN rp.category = 'actress' AND rp.person_rank = 3 THEN rp.person_name END) AS actress3,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 1 THEN rp.person_name END) AS actor1,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 2 THEN rp.person_name END) AS actor2,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 3 THEN rp.person_name END) AS actor3,
    MAX(CASE WHEN rp.category = 'actor' AND rp.person_rank = 4 THEN rp.person_name END) AS actor4,
    MAX(CASE WHEN rp.category = 'producer' AND rp.person_rank = 1 THEN rp.person_name END) AS producer1,
    MAX(CASE WHEN rp.category = 'producer' AND rp.person_rank = 2 THEN rp.person_name END) AS producer2
FROM movie_ratings mr
JOIN movie_basics mb ON mr.movie_id = mb.movie_id
LEFT JOIN ranked_directors rd ON mr.movie_id = rd.movie_id
LEFT JOIN ranked_principals rp ON mr.movie_id = rp.movie_id
WHERE mr.numvotes >= 30
GROUP BY 
    mr.movie_id, 
    mb.primary_title, 
    mr.averagerating, 
    mr.numvotes, 
    mb.start_year, 
    mb.runtime_minutes, 
    mb.genres
ORDER BY -mr.numvotes

"""
pd.read_sql(query201, conn)

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director1,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
0,tt1375666,Inception,8.8,1841066,2010,148.0,"Action,Adventure,Sci-Fi",Christopher Nolan,None,Ellen Page,None,None,Leonardo DiCaprio,Joseph Gordon-Levitt,Ken Watanabe,None,Emma Thomas,None
1,tt1345836,The Dark Knight Rises,8.4,1387769,2012,164.0,"Action,Thriller",Christopher Nolan,None,Anne Hathaway,None,None,Gary Oldman,Christian Bale,Tom Hardy,None,Charles Roven,Emma Thomas
2,tt0816692,Interstellar,8.6,1299334,2014,169.0,"Adventure,Drama,Sci-Fi",Christopher Nolan,None,Anne Hathaway,Jessica Chastain,Mackenzie Foy,Matthew McConaughey,None,None,None,Lynda Obst,Emma Thomas
3,tt1853728,Django Unchained,8.4,1211405,2012,165.0,"Drama,Western",Quentin Tarantino,None,Kerry Washington,None,None,Leonardo DiCaprio,Jamie Foxx,Christoph Waltz,None,Reginald Hudlin,Pilar Savone
4,tt0848228,The Avengers,8.1,1183655,2012,143.0,"Action,Adventure,Sci-Fi",Joss Whedon,None,Scarlett Johansson,None,None,Robert Downey Jr.,Chris Evans,Jeremy Renner,None,Kevin Feige,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43735,tt9378760,Sarah Millican: Control Enthusiast Live,6.9,30,2018,82.0,Comedy,Brian Klein,None,None,None,None,None,None,None,None,None,None
43736,tt9442146,Hüddam 2,5.3,30,2019,92.0,"Drama,Horror,Thriller",Utku Uçar,None,Seyda Ipek Baykal,Ayyildiz Beslen,None,Hakan Erkuran,Can Beslen,None,None,Alpaslan Abdurrahman,None
43737,tt9598566,Ave Maria,7.3,30,2018,74.0,Drama,Vipin Radhakrishnan,None,Mekha Rajan,None,None,Ashand Raju,Reshma Malayath,Sabumon Abdusamad,None,None,None
43738,tt9613316,Frances Ferguson,6.8,30,2019,74.0,Comedy,Bob Byington,None,Jennifer Prediger,Kaley Wheless,None,Nick Offerman,Martin Starr,None,None,Zefrey Throwell,None


In [268]:
imdb_df = pd.read_sql(query201, conn)
imdb_df.head()

,imdb_movie_id,primary_title,average_rating,numvotes,start_year,runtime_minutes,genres,director1,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
0,tt1375666,Inception,8.8,1841066,2010,148.0,"Action,Adventure,Sci-Fi",Christopher Nolan,None,Ellen Page,None,None,Leonardo DiCaprio,Joseph Gordon-Levitt,Ken Watanabe,None,Emma Thomas,None
1,tt1345836,The Dark Knight Rises,8.4,1387769,2012,164.0,"Action,Thriller",Christopher Nolan,None,Anne Hathaway,None,None,Gary Oldman,Christian Bale,Tom Hardy,None,Charles Roven,Emma Thomas
2,tt0816692,Interstellar,8.6,1299334,2014,169.0,"Adventure,Drama,Sci-Fi",Christopher Nolan,None,Anne Hathaway,Jessica Chastain,Mackenzie Foy,Matthew McConaughey,None,None,None,Lynda Obst,Emma Thomas
3,tt1853728,Django Unchained,8.4,1211405,2012,165.0,"Drama,Western",Quentin Tarantino,None,Kerry Washington,None,None,Leonardo DiCaprio,Jamie Foxx,Christoph Waltz,None,Reginald Hudlin,Pilar Savone
4,tt0848228,The Avengers,8.1,1183655,2012,143.0,"Action,Adventure,Sci-Fi",Joss Whedon,None,Scarlett Johansson,None,None,Robert Downey Jr.,Chris Evans,Jeremy Renner,None,Kevin Feige,None


In [ ]:
#######################################

In [53]:
query11 = """ SELECT * FROM directors LIMIT 5"""
pd.read_sql(query11, conn)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [112]:
query31 = """ SELECT COUNT(*) FROM (SELECT DISTINCT * FROM directors)"""
pd.read_sql(query31, conn)

,COUNT(*)
0,163535


In [54]:
query12 = """ SELECT * FROM writers LIMIT 5"""
pd.read_sql(query12, conn)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087


In [105]:
# Read the CSV file
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')

# Display the first few rows of the DataFrame
print(bom_df.head(15))

                                                title studio  domestic_gross  \
0                                         Toy Story 3     BV     415000000.0   
1                          Alice in Wonderland (2010)     BV     334200000.0   
2         Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                           Inception     WB     292600000.0   
4                                 Shrek Forever After   P/DW     238700000.0   
5                          The Twilight Saga: Eclipse   Sum.     300500000.0   
6                                          Iron Man 2   Par.     312400000.0   
7                                             Tangled     BV     200800000.0   
8                                       Despicable Me   Uni.     251500000.0   
9                            How to Train Your Dragon   P/DW     217600000.0   
10                         Clash of the Titans (2010)     WB     163200000.0   
11  The Chronicles of Narnia: The Voyage

In [ ]:
###################

In [164]:
# Read the CSV file
budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# Display the first few rows of the DataFrame
print(budgets_df.tail(20))

      id  release_date                            movie production_budget  \
5762  63  Apr 11, 1997                   Pink Flamingos           $12,000   
5763  64  Apr 28, 2006         Grip: A Criminal's Story           $12,000   
5764  65  Dec 31, 2007                      Tin Can Man           $12,000   
5765  66   Mar 9, 2001                          Dayereh           $10,000   
5766  67  Apr 28, 2006                            Clean           $10,000   
5767  68   Jul 6, 2001                             Cure           $10,000   
5768  69  May 28, 2004                   On the Downlow           $10,000   
5769  70   Apr 1, 1996                             Bang           $10,000   
5770  71  Aug 14, 2008  The Rise and Fall of Miss Thang           $10,000   
5771  72  May 19, 2015                 Family Motocross           $10,000   
5772  73  Jan 13, 2012                        Newlyweds            $9,000   
5773  74  Feb 26, 1993                      El Mariachi            $7,000   

In [135]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [163]:
budgets_df['movie'].nunique()

5698

In [169]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [171]:
# convert release_dat column to datetime
budgets_df['release_date'] = pd.to_datetime(budgets_df['release_date'], format='%b %d, %Y')

In [172]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   object        
 4   domestic_gross     5782 non-null   object        
 5   worldwide_gross    5782 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 271.2+ KB


In [173]:
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [174]:
# filter out the rows before 2010
budgets_df = budgets_df[budgets_df['release_date'] >= '2010-01-01']

In [175]:
budgets_df['movie'].nunique()

2190

In [183]:
xx = budgets_df.sort_values(by='domestic_gross')
xx.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
2915,16,2019-07-12,Crawl,"$17,000,000",$0,$0
4381,82,2012-10-26,Mientras duermes,"$5,000,000",$0,"$9,109,597"
4384,85,2014-12-05,Dying of the Light,"$5,000,000",$0,"$74,639"
4385,86,2016-09-02,Yoga Hosers,"$5,000,000",$0,"$22,958"
4388,89,2010-12-31,Psych 9,"$5,000,000",$0,$0


In [177]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2194 entries, 1 to 5780
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2194 non-null   int64         
 1   release_date       2194 non-null   datetime64[ns]
 2   movie              2194 non-null   object        
 3   production_budget  2194 non-null   object        
 4   domestic_gross     2194 non-null   object        
 5   worldwide_gross    2194 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 120.0+ KB


In [178]:
duplicated_movies = budgets_df[budgets_df['movie'].duplicated(keep=False)]
dd = duplicated_movies.sort_values(by='release_date')
dd

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5009,10,2010-04-09,The Square,"$1,900,000","$406,216","$740,932"
38,39,2010-05-14,Robin Hood,"$210,000,000","$105,487,148","$322,459,006"
5330,31,2012-12-31,Trance,"$950,000",$0,$0
5351,52,2012-12-31,Snitch,"$850,000",$0,$0
3025,26,2013-02-22,Snitch,"$15,000,000","$42,930,462","$57,907,734"
2970,71,2013-04-05,Trance,"$16,000,000","$2,322,593","$22,594,052"
5099,100,2013-10-25,The Square,"$1,500,000","$124,244","$176,262"
408,9,2018-11-21,Robin Hood,"$99,000,000","$30,824,628","$84,747,441"


In [186]:
# add a new column with just the year
budgets_df['release_year'] = budgets_df['release_date'].dt.year
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220",2015


In [ ]:
#####################################################################################

In [ ]:
# record_linkage

In [283]:
# Normalize titles by converting to lowercase and stripping spaces
budgets_df['movie'] = budgets_df['movie'].str.lower().str.strip()
imdb_df['primary_title'] = imdb_df['primary_title'].str.lower().str.strip()

In [211]:
# Create an indexer object for record linkage
indexer = recordlinkage.Index()

# Block on year/start_year to only compare movies with the same release year
indexer.block(left_on='release_year', right_on='start_year')

# Generate candidate pairs
candidate_pairs = indexer.index(budgets_df, imdb_df)

# Inspect candidate pairs
print(candidate_pairs)

MultiIndex([(   1, 35554),
            (   1, 35555),
            (   1, 35556),
            (   1, 35557),
            (   1, 35558),
            (   1, 35559),
            (   1, 35560),
            (   1, 35561),
            (   1, 35562),
            (   1, 35563),
            ...
            (5780, 21269),
            (5780, 21270),
            (5780, 21271),
            (5780, 21272),
            (5780, 21273),
            (5780, 21274),
            (5780, 21275),
            (5780, 21276),
            (5780, 21277),
            (5780, 21278)],
           length=10100251)


In [212]:
# Create a Compare object
compare = recordlinkage.Compare()

# Compare movie titles with a fuzzy string matching algorithm (Levenshtein or Jaro-Winkler)
compare.string('movie', 'primary_title', method='jarowinkler', threshold=0.85, label='title_match')

# Compare years exactly
compare.exact('release_year', 'start_year', label='year_match')

# Compute the features for the candidate pairs
features = compare.compute(candidate_pairs, budgets_df, imdb_df)

# Display features (each row represents a candidate pair, with columns for comparison metrics)
print(features)

            title_match  year_match
1    35554          0.0           1
     35555          0.0           1
     35556          0.0           1
     35557          0.0           1
     35558          0.0           1
...                 ...         ...
5780 21274          0.0           1
     21275          0.0           1
     21276          0.0           1
     21277          0.0           1
     21278          0.0           1

[10100251 rows x 2 columns]


In [213]:
# Select matches where both year matches and title similarity score is high (above threshold)
matches = features[(features['year_match'] == 1) & (features['title_match'] >= 0.85)]

# Get the indices of matched rows from the original DataFrames
matched_pairs = matches.index

# View the matched pairs
matched_budgets_df = budgets_df.loc[matched_pairs.get_level_values(0)]
matched_imdb_df = imdb_df.loc[matched_pairs.get_level_values(1)]

# Display the matched results
print(matched_budgets_df)

      id release_date                                        movie  \
1      2   2011-05-20  pirates of the caribbean: on stranger tides   
2      3   2019-06-07                                 dark phoenix   
2      3   2019-06-07                                 dark phoenix   
3      4   2015-05-01                      avengers: age of ultron   
4      5   2017-12-15            star wars ep. viii: the last jedi   
...   ..          ...                                          ...   
5761  62   2014-12-31                         stories of our lives   
5771  72   2015-05-19                             family motocross   
5771  72   2015-05-19                             family motocross   
5771  72   2015-05-19                             family motocross   
5771  72   2015-05-19                             family motocross   

     production_budget domestic_gross worldwide_gross  release_year  
1         $410,600,000   $241,063,875  $1,045,663,875          2011  
2         $350,000,

In [214]:
print(matched_imdb_df)

      imdb_movie_id                                primary_title  \
35796     tt1298650  pirates of the caribbean: on stranger tides   
151       tt4082314                                   dark sense   
309       tt6565702                                 dark phoenix   
16627     tt2395427                      avengers: age of ultron   
5851      tt2527336                     star wars: the last jedi   
...             ...                                          ...   
25524     tt3973612                         stories of our lives   
16316     tt1700807                                family secret   
16495     tt2187037                                       family   
20860     tt5133038                                family member   
20976     tt5212256                           family commitments   

       average_rating  numvotes  start_year  runtime_minutes  \
35796             6.6    447624        2011            136.0   
151               5.3        82        2019            

In [ ]:
###

In [203]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2194 entries, 1 to 5780
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2194 non-null   int64         
 1   release_date       2194 non-null   datetime64[ns]
 2   movie              2194 non-null   object        
 3   production_budget  2194 non-null   object        
 4   domestic_gross     2194 non-null   object        
 5   worldwide_gross    2194 non-null   object        
 6   release_year       2194 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(4)
memory usage: 193.1+ KB


In [204]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43740 entries, 0 to 43739
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   imdb_movie_id    43740 non-null  object 
 1   primary_title    43740 non-null  object 
 2   average_rating   43740 non-null  float64
 3   numvotes         43740 non-null  int64  
 4   start_year       43740 non-null  int64  
 5   runtime_minutes  41466 non-null  float64
 6   genres           43619 non-null  object 
 7   director1        43528 non-null  object 
 8   director2        4352 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 3.0+ MB


In [205]:
budgets_df['release_year'] = budgets_df['release_year'].astype('int64')

In [206]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2194 entries, 1 to 5780
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2194 non-null   int64         
 1   release_date       2194 non-null   datetime64[ns]
 2   movie              2194 non-null   object        
 3   production_budget  2194 non-null   object        
 4   domestic_gross     2194 non-null   object        
 5   worldwide_gross    2194 non-null   object        
 6   release_year       2194 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 201.7+ KB


In [284]:
# Check for missing values in each column
budgets_df.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
release_year         0
index                0
dtype: int64

In [285]:
imdb_df.isnull().sum()

imdb_movie_id          0
primary_title          0
average_rating         0
numvotes               0
start_year             0
runtime_minutes     2274
genres               121
director1            212
director2          39388
actress1           10698
actress2           24479
actress3           37581
actor1              6482
actor2             11524
actor3             24337
actor4             37408
producer1          12035
producer2          27328
index                  0
dtype: int64

In [ ]:
##############################################################################

In [269]:
# Create an indexer and define the comparison criteria
indexer = recordlinkage.Index()
indexer.block(left_on='release_year', right_on='start_year')
candidate_links = indexer.index(budgets_df, imdb_df)

compare = recordlinkage.Compare()
compare.exact('movie', 'primary_title', label='title')
compare.exact('release_year', 'start_year', label='year')
features = compare.compute(candidate_links, budgets_df, imdb_df)


In [270]:
# Filter matches based on a threshold
matches = features[features.sum(axis=1) > 1]

In [271]:
# Merge the data frames based on the matches
budgets_df['index'] = budgets_df.index
imdb_df['index'] = imdb_df.index

matches.reset_index(inplace=True)
merged_df = pd.merge(budgets_df, matches, left_on='index', right_on='level_0')
merged_df = pd.merge(merged_df, imdb_df, left_on='level_1', right_on='index')

# Drop unnecessary columns and duplicates
merged_df.drop(columns=['index_x', 'index_y', 'level_0', 'level_1'], inplace=True)
merged_df.drop_duplicates(inplace=True)

In [272]:
merged_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,title,year,imdb_movie_id,...,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
0,22,2013-04-12,42,"$31,000,000","$95,020,213","$97,470,701",2013,1,1,tt0453562,...,None,Nicole Beharie,None,None,Harrison Ford,T.R. Knight,Chadwick Boseman,None,Thomas Tull,None
1,93,2014-03-21,50 to 1,"$10,000,000","$1,069,454","$1,069,454",2014,1,1,tt1777595,...,None,Madelyn Deutch,None,None,Skeet Ulrich,William Devane,Christian Kane,None,None,None
2,73,2011-09-30,50/50,"$8,000,000","$35,016,118","$41,334,735",2011,1,1,tt1306980,...,None,Bryce Dallas Howard,Anna Kendrick,None,Joseph Gordon-Levitt,Seth Rogen,None,None,Ben Karlin,Evan Goldberg
3,85,2016-10-21,31,"$1,500,000","$779,820","$922,727",2016,1,1,tt3835080,...,None,Sheri Moon Zombie,None,None,Malcolm McDowell,Richard Brake,Jeff Daniel Phillips,None,Mike Elliott,Andy Gould


In [273]:
merged_df.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
release_year         0
title                0
year                 0
imdb_movie_id        0
primary_title        0
average_rating       0
numvotes             0
start_year           0
runtime_minutes      0
genres               0
director1            0
director2            4
actress1             0
actress2             3
actress3             4
actor1               0
actor2               0
actor3               1
actor4               4
producer1            1
producer2            2
dtype: int64

In [274]:
####

In [286]:
# years have to match exactly but not movie titles
# Create an indexer and define the comparison criteria
indexer = recordlinkage.Index()
indexer.block(left_on='release_year', right_on='start_year')
candidate_links = indexer.index(budgets_df, imdb_df)

compare = recordlinkage.Compare()
compare.exact('release_year', 'start_year', label='year')
compare.string('movie', 'primary_title', method='jarowinkler', threshold=0.95, label='title')
features = compare.compute(candidate_links, budgets_df, imdb_df)

In [287]:
# Filter matches based on a threshold
matches = features[features.sum(axis=1) > 1.5]

In [288]:
# Merge the data frames based on the matches
budgets_df['index'] = budgets_df.index
imdb_df['index'] = imdb_df.index

matches.reset_index(inplace=True)
merged_df = pd.merge(budgets_df, matches, left_on='index', right_on='level_0')
merged_df = pd.merge(merged_df, imdb_df, left_on='level_1', right_on='index')

# Drop unnecessary columns and duplicates
merged_df.drop(columns=['index_x', 'index_y', 'level_0', 'level_1'], inplace=True)
merged_df.drop_duplicates(inplace=True)
merged_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,year,title,imdb_movie_id,...,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
0,2,2011-05-20,pirates of the caribbean: on stranger tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,1,1.0,tt1298650,...,None,Penélope Cruz,None,None,Johnny Depp,Geoffrey Rush,Ian McShane,None,None,None
1,3,2019-06-07,dark phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,1,1.0,tt6565702,...,None,Jennifer Lawrence,None,None,Nicholas Hoult,James McAvoy,Michael Fassbender,None,None,None
2,4,2015-05-01,avengers: age of ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,1,1.0,tt2395427,...,None,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Kevin Feige,None
3,7,2018-04-27,avengers: infinity war,"$300,000,000","$678,815,482","$2,048,134,200",2018,1,1.0,tt4154756,...,Joe Russo,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,None,None
4,9,2017-11-17,justice league,"$300,000,000","$229,024,295","$655,945,209",2017,1,1.0,tt0974015,...,None,Gal Gadot,None,None,Ben Affleck,Jason Momoa,Ezra Miller,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,35,2013-10-25,her cry: la llorona investigation,"$35,000",$0,$0,2013,1,1.0,tt2469216,...,None,Nichole Ceballos,None,None,Ron Gelner,Everardo Guzman,James Ezrin,None,Hai Nguyen,None
1596,49,2015-09-01,exeter,"$25,000",$0,"$489,792",2015,1,1.0,tt1945044,...,None,Brittany Curran,None,None,Stephen Lang,Kevin Chapman,Kelly Blatz,None,Trent Othick,Michael Corso
1597,52,2015-12-01,dutch kills,"$25,000",$0,$0,2015,1,1.0,tt2759066,...,None,Celestine Rae,None,None,R.L. Mann,Mikaal Bates,Tama Filianga,None,Liam Le Guillou,None
1598,59,2011-11-25,the ridges,"$17,300",$0,$0,2011,1,1.0,tt1781935,...,None,Alana Kaniewski,Robbie Barnes,None,Ryan Dunk,Robert Grant,None,None,None,None


In [289]:
# rearrange columns
columns = list(merged_df.columns)

# Move 'primary_title' next to 'movie'
columns.insert(columns.index('movie') + 1, columns.pop(columns.index('primary_title')))
merged_df = merged_df[columns]
merged_df

,id,release_date,movie,primary_title,production_budget,domestic_gross,worldwide_gross,release_year,year,title,...,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
0,2,2011-05-20,pirates of the caribbean: on stranger tides,pirates of the caribbean: on stranger tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,1,1.0,...,None,Penélope Cruz,None,None,Johnny Depp,Geoffrey Rush,Ian McShane,None,None,None
1,3,2019-06-07,dark phoenix,dark phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,1,1.0,...,None,Jennifer Lawrence,None,None,Nicholas Hoult,James McAvoy,Michael Fassbender,None,None,None
2,4,2015-05-01,avengers: age of ultron,avengers: age of ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,1,1.0,...,None,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Kevin Feige,None
3,7,2018-04-27,avengers: infinity war,avengers: infinity war,"$300,000,000","$678,815,482","$2,048,134,200",2018,1,1.0,...,Joe Russo,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,None,None
4,9,2017-11-17,justice league,justice league,"$300,000,000","$229,024,295","$655,945,209",2017,1,1.0,...,None,Gal Gadot,None,None,Ben Affleck,Jason Momoa,Ezra Miller,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,35,2013-10-25,her cry: la llorona investigation,her cry: la llorona investigation,"$35,000",$0,$0,2013,1,1.0,...,None,Nichole Ceballos,None,None,Ron Gelner,Everardo Guzman,James Ezrin,None,Hai Nguyen,None
1596,49,2015-09-01,exeter,exeter,"$25,000",$0,"$489,792",2015,1,1.0,...,None,Brittany Curran,None,None,Stephen Lang,Kevin Chapman,Kelly Blatz,None,Trent Othick,Michael Corso
1597,52,2015-12-01,dutch kills,dutch kills,"$25,000",$0,$0,2015,1,1.0,...,None,Celestine Rae,None,None,R.L. Mann,Mikaal Bates,Tama Filianga,None,Liam Le Guillou,None
1598,59,2011-11-25,the ridges,the ridges,"$17,300",$0,$0,2011,1,1.0,...,None,Alana Kaniewski,Robbie Barnes,None,Ryan Dunk,Robert Grant,None,None,None,None


In [290]:
# Filter rows where 'movie' does not equal 'primary_title'
non_exact_matches = merged_df[merged_df['movie'] != merged_df['primary_title']]

# Set display options to show all rows
pd.set_option('display.max_rows', None)

non_exact_matches

,id,release_date,movie,primary_title,production_budget,domestic_gross,worldwide_gross,release_year,year,title,...,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
33,49,2017-05-05,guardians of the galaxy vol 2,guardians of the galaxy vol. 2,"$200,000,000","$389,813,101","$862,316,233",2017,1,1.0,...,None,Zoe Saldana,None,None,Vin Diesel,Chris Pratt,Dave Bautista,None,None,None
65,92,2018-07-27,mission: impossibleâfallout,mission: impossible - fallout,"$178,000,000","$220,159,104","$787,456,552",2018,1,1.0,...,None,None,None,None,Tom Cruise,Ving Rhames,Henry Cavill,Simon Pegg,J.J. Abrams,Jake Myers
97,32,2014-11-05,interstellar,interstelar,"$165,000,000","$188,017,894","$666,379,375",2014,1,1.0,...,Vladislav Krivenchuk,None,None,None,Viktor Murzikov,Sergey A.,Vladislav Krivenchuk,Dmitriy Khmelyov,None,None
103,40,2013-05-24,fast and furious 6,fast & furious 6,"$160,000,000","$238,679,850","$789,300,444",2013,1,1.0,...,None,Michelle Rodriguez,None,None,Vin Diesel,Dwayne Johnson,Paul Walker,None,Neal H. Moritz,Clayton Townsend
110,49,2015-07-01,terminator: genisys,terminator genisys,"$155,000,000","$89,760,956","$432,150,894",2015,1,1.0,...,None,Emilia Clarke,None,None,Arnold Schwarzenegger,Jason Clarke,Jai Courtney,None,David Ellison,None
122,76,2019-05-10,pokã©mon: detective pikachu,pokémon detective pikachu,"$150,000,000","$139,507,806","$411,258,433",2019,1,1.0,...,None,Kathryn Newton,None,None,Ryan Reynolds,Bill Nighy,Justice Smith,None,None,None
142,25,2012-11-16,"the twilight saga: breaking dawn, part 2",the twilight saga: breaking dawn - part 2,"$136,200,000","$292,324,737","$829,724,737",2012,1,1.0,...,None,Kristen Stewart,None,None,Peter Facinelli,Taylor Lautner,Robert Pattinson,None,Wyck Godfrey,Karen Rosenfelt
145,27,2015-12-25,the revenant,the event,"$135,000,000","$183,637,894","$532,938,302",2015,1,1.0,...,None,None,None,None,None,None,None,None,Maria Baker-Choustova,None
166,57,2011-11-18,"the twilight saga: breaking dawn, part 1",the twilight saga: breaking dawn - part 1,"$127,500,000","$281,287,133","$689,420,051",2011,1,1.0,...,None,Kristen Stewart,None,None,Gil Birmingham,Taylor Lautner,Robert Pattinson,None,Wyck Godfrey,Karen Rosenfelt
169,61,2011-07-15,harry potter and the deathly hallows: part ii,harry potter and the deathly hallows: part 2,"$125,000,000","$381,193,157","$1,341,693,157",2011,1,1.0,...,None,Emma Watson,None,None,Michael Gambon,Rupert Grint,Daniel Radcliffe,None,David Barron,David Heyman


In [291]:
# Reset display options to default
pd.reset_option('display.max_rows')

In [294]:
# remove movies with poorly matched titles by index
poor_match = [97, 145, 447, 495, 563, 853, 946, 1160, 1240, 1242]
merged_df = merged_df.drop(poor_match)

In [296]:
# set imdb_movie_id column as the index
merged_df.set_index('imdb_movie_id', inplace=True)

In [298]:
merged_df.head()

,id,release_date,movie,primary_title,production_budget,domestic_gross,worldwide_gross,release_year,year,title,...,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
imdb_movie_id,,,,,,,,,,,,,,,,,,,,,
tt1298650,2,2011-05-20,pirates of the caribbean: on stranger tides,pirates of the caribbean: on stranger tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,1,1.0,...,None,Penélope Cruz,None,None,Johnny Depp,Geoffrey Rush,Ian McShane,None,None,None
tt6565702,3,2019-06-07,dark phoenix,dark phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,1,1.0,...,None,Jennifer Lawrence,None,None,Nicholas Hoult,James McAvoy,Michael Fassbender,None,None,None
tt2395427,4,2015-05-01,avengers: age of ultron,avengers: age of ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,1,1.0,...,None,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Kevin Feige,None
tt4154756,7,2018-04-27,avengers: infinity war,avengers: infinity war,"$300,000,000","$678,815,482","$2,048,134,200",2018,1,1.0,...,Joe Russo,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,None,None
tt0974015,9,2017-11-17,justice league,justice league,"$300,000,000","$229,024,295","$655,945,209",2017,1,1.0,...,None,Gal Gadot,None,None,Ben Affleck,Jason Momoa,Ezra Miller,None,None,None


In [300]:
merged_df.columns

Index(['id', 'release_date', 'movie', 'primary_title', 'production_budget',
       'domestic_gross', 'worldwide_gross', 'release_year', 'year', 'title',
       'average_rating', 'numvotes', 'start_year', 'runtime_minutes', 'genres',
       'director1', 'director2', 'actress1', 'actress2', 'actress3', 'actor1',
       'actor2', 'actor3', 'actor4', 'producer1', 'producer2'],
      dtype='object')

In [302]:
col_to_remove = ['id', 'movie', 'release_year', 'year', 'title', 'start_year', 'runtime_minutes']
merged_df = merged_df.drop(col_to_remove, axis=1)

In [303]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1590 entries, tt1298650 to tt3973612
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1590 non-null   datetime64[ns]
 1   primary_title      1590 non-null   object        
 2   production_budget  1590 non-null   object        
 3   domestic_gross     1590 non-null   object        
 4   worldwide_gross    1590 non-null   object        
 5   average_rating     1590 non-null   float64       
 6   numvotes           1590 non-null   int64         
 7   genres             1590 non-null   object        
 8   director1          1589 non-null   object        
 9   director2          139 non-null    object        
 10  actress1           1383 non-null   object        
 11  actress2           730 non-null    object        
 12  actress3           177 non-null    object        
 13  actor1             1534 non-null   object        
 14  

In [306]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1590 entries, tt1298650 to tt3973612
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1590 non-null   datetime64[ns]
 1   primary_title      1590 non-null   object        
 2   production_budget  1590 non-null   object        
 3   domestic_gross     1590 non-null   object        
 4   worldwide_gross    1590 non-null   object        
 5   average_rating     1590 non-null   float64       
 6   numvotes           1590 non-null   int64         
 7   genres             1590 non-null   object        
 8   director1          1589 non-null   object        
 9   director2          139 non-null    object        
 10  actress1           1383 non-null   object        
 11  actress2           730 non-null    object        
 12  actress3           177 non-null    object        
 13  actor1             1534 non-null   object        
 14  

In [305]:
merged_df.head()

,release_date,primary_title,production_budget,domestic_gross,worldwide_gross,average_rating,numvotes,genres,director1,director2,actress1,actress2,actress3,actor1,actor2,actor3,actor4,producer1,producer2
imdb_movie_id,,,,,,,,,,,,,,,,,,,
tt1298650,2011-05-20,pirates of the caribbean: on stranger tides,"$410,600,000","$241,063,875","$1,045,663,875",6.6,447624,"Action,Adventure,Fantasy",Rob Marshall,None,Penélope Cruz,None,None,Johnny Depp,Geoffrey Rush,Ian McShane,None,None,None
tt6565702,2019-06-07,dark phoenix,"$350,000,000","$42,762,350","$149,762,350",6.0,24451,"Action,Adventure,Sci-Fi",Simon Kinberg,None,Jennifer Lawrence,None,None,Nicholas Hoult,James McAvoy,Michael Fassbender,None,None,None
tt2395427,2015-05-01,avengers: age of ultron,"$330,600,000","$459,005,868","$1,403,013,963",7.3,665594,"Action,Adventure,Sci-Fi",Joss Whedon,None,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,Kevin Feige,None
tt4154756,2018-04-27,avengers: infinity war,"$300,000,000","$678,815,482","$2,048,134,200",8.5,670926,"Action,Adventure,Sci-Fi",Anthony Russo,Joe Russo,None,None,None,Robert Downey Jr.,Chris Evans,Mark Ruffalo,Chris Hemsworth,None,None
tt0974015,2017-11-17,justice league,"$300,000,000","$229,024,295","$655,945,209",6.5,329135,"Action,Adventure,Fantasy",Zack Snyder,None,Gal Gadot,None,None,Ben Affleck,Jason Momoa,Ezra Miller,None,None,None
